# Multi agent systems with tools

In [8]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages,AnyMessage

from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode, tools_condition
from IPython.display import Image, display
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import Runnable
from langgraph.graph import StateGraph, MessagesState, START, END
from langchain_core.messages import ToolMessage
from enum import Enum
from langchain_core.pydantic_v1 import BaseModel,Field

In [10]:
# LLM

llm = ChatOpenAI(model = "gpt-4o", temperature = 0)

In [11]:
# STATE

class State(TypedDict):
    messages: Annotated[list,add_messages]

In [12]:

class agents(str,Enum):
    multi_agent = "multi_agent"
    numeric_agent = "numeric_agent"
    story_agent = "story_agent"
    connecting_agent = "connecting_agent"
    connecting_num_agent = "connecting_num_agent"
    connecting_story_agent = "connecting_story_agent"
    checking_tools = "checking_tools"
    leave_skill = "leave_skill"

In [13]:
# ENTRY NODE FOR CONNECTING_AGENT(entry_node is a tool message)

def create_entry_node(assistant_name:str, new_dialogue_state: str) -> str:
    def entry_node(state: State)->dict:
        tool_call_id = state['messages'][-1].tool_calls[0]["id"]
        return {
            "messages":[ToolMessage(content = 
            f"""The assistant is now the {assistant_name}. Reflect on the above conversation
            between the host assistant and the user.The user's intent is unsatisfied.
            Use the provided tools to assist the user. Remember, you are {assistant_name},
            If the user is trying to perform operations that are not supported by you
            then call the CompleteOrEscalate. If the user changes their mind or needs
            help for other tasks, call the CompleteOrEscalate function to let the primary
            host assistant take control.Do not mention who you are - just act as the
            proxy for the assistant.""",

                        tool_call_id = tool_call_id,)],
            "dialog_state" : pop_dialog_state
            
        }
    return entry_node

In [14]:
class CompleteOrEscalate(BaseModel):
    """A tool to mark the current task as completed and/or to escalate
    control of the dialog to the main assistant, who can re-route the dialog
    based on the user's needs."""

    cancel: bool = True
    reason: str

    class Config:
        schema_extra = {
            "example": {
                "cancel": True,
                "reason": "User changed their mind about the current task.",
            },
            "example 2": {
                "cancel": True,
                "reason": "I have fully completed the task.",
            },
        }

# This node will be shared for exiting all specialized assistants
def pop_dialog_state(state: State) -> dict:
    """Pop the dialog stack and return to the main assistant.

    This lets the full graph explicitly track the dialog flow and delegate
    control to specific sub-graphs.
    """
    messages = []
    if state["messages"][-1].tool_calls:
        messages.append(
            ToolMessage(
                content="""Resuming dialog with the host assistant. Please
reflect on the past conversation and assist the user as needed.""",
                tool_call_id=state["messages"][-1].tool_calls[0]["id"],
            )
        )
    return {
        "dialog_state": "pop",
        "messages": messages,
    }

In [15]:
#TOOLS

@tool
def check() -> str:
    """
    Call this tool before responding to the user to check if your response is valid.
    """
    return "Your response is valid"

class ToNumericAssistant(BaseModel):
    """
    Transfers work to a specialized to handle numeric operations. This tool does not perform any operation that are not related to numbers
    """
    request: str = Field(
        description="""
        Any additional information or requests from the user
        regarding the numeric operation."""
    )
    reason: str = Field(
        description="Reason why this tool is used for the operation"
    )


class ToStoryAssistant(BaseModel):
    """
    Transfers work to a specialized to handle creation of stories. This tool does not perform any operation that are not related to story.
    """
    request: str = Field(
        description="""
        Any additional information or requests from the user
        regarding the story creation."""
    )
    reason: str = Field(
        description="Reason why this tool is used for the operation"
    )


## NUMERIC AGENT

In [16]:
#DEFINING AGENT
def numeric_agent(state:State):
    response = numeric_chain(llm).invoke({"messages":state["messages"]})
    return {"messages":[response]}


In [17]:
# TOOLS

@tool
def numeric_tool(query: str) -> str:
    """You are an AI assistant for solving numeric problems."""
    operators = """
    
    Evaluates a numeric expression. Supports basic arithmetic and
    some advanced operations like factorial, exponentiation, and modulo.
    
    Supported operations:
    - Addition (+)
    - Subtraction (-)
    - Multiplication (*)
    - Division (/)
    - Floor division (//)
    - Modulus (%)
    - Exponentiation (** or ^)
    - Factorial (!) 
    
    Example usage:
    - "5 + 3 * 2"
    - "10 / 2"
    - "4!"
    """
    

tools = [numeric_tool]
tool_node = ToolNode(tools)
    

In [18]:
#PROMPTS

numeric_agent_system_prompt = """

You are an AI assistant and your job is to solve numeric operations.
Don't provide with any other activities.

You are provided with numeric_tool to perform operations.
Don't use anything outside this tool.
Do not try to be conversational

Always use CompleteOrEscalate tool after completing. If you are calling CompleteOrEscalate then explain the reason
If the user needs help, and you did not find appropriate function after searching, then 
CompleteOrEscalate the dialog to the host assistant. Do not waste the users time. Do not make up invalid tools or functions.
"""


numeric_agent_prompt = ChatPromptTemplate.from_messages([
        ("system", numeric_agent_system_prompt),
        ("placeholder", "{messages}"),])


In [19]:
# CHAINS
def numeric_chain(llm):
    return ( numeric_agent_prompt
            | llm.bind_tools(                
                [
                    numeric_tool,CompleteOrEscalate
                ]))

In [20]:
# ROUTER
def numeric_route(state: State):
    """
    Use in the conditional_edge to route to the ToolNode if the last message
    has tool calls. Otherwise, route to the end.
    """
    route = tools_condition(state)
    if route == END:
        return agents.leave_skill
    tool_calls = state["messages"][-1].tool_calls
    print(tool_calls)
    if tool_calls:
        tool_name = tool_calls[0]["name"]
        if tool_name == "numeric_tool":
            print(10 * "#")
            print(tool_name)
            print(10 * "#")
            return 'tools'
        elif tool_name == CompleteOrEscalate.__name__:
            return agents.leave_skill
    else:
        raise ValueError("invalid tool call")

In [21]:
# workflow (to go to one agent to another agent we need connectind_agent(tool message))
def numeric_builder(graph_builder:StateGraph, llm:Runnable) -> StateGraph:
    graph_builder.add_node(agents.connecting_num_agent,create_entry_node("Tool message before going to numeric_agent",agents.numeric_agent))
    graph_builder.add_node(agents.numeric_agent, numeric_agent)
    graph_builder.add_edge(agents.connecting_num_agent,agents.numeric_agent)
    graph_builder.add_conditional_edges(agents.numeric_agent,numeric_route)
    graph_builder.add_node("tools", tool_node)
    graph_builder.add_edge("tools", "numeric_agent")
    return graph_builder

## STORY AGENT

In [22]:
# DEFINING AGENT
def story_agent(state:State):
    response = story_chain(llm).invoke({"messages":state["messages"]})
    return {"messages":[response]}

In [23]:
#PROMPTS

story_agent_system_prompt = """
You are an assistant and your task is to generate a unique love story with whatever query users give.
You should only accept the queries related to generation of stories, nothing else should be accepted.
Don't perform any other activities.Only give stories. 
Generate a story of 5 lines.
"""

story_agent_prompt = ChatPromptTemplate.from_messages([
        ("system", story_agent_system_prompt),
        ("placeholder", "{messages}"),])

In [24]:
# CHAINS
def story_chain(llm):
    return ( story_agent_prompt|llm)

In [25]:
# workflow
def story_builder(graph_builder:StateGraph, llm:Runnable) -> StateGraph:
    graph_builder.add_node(agents.connecting_story_agent,create_entry_node("Tool message before going to story_agent",agents.story_agent))
    graph_builder.add_node(agents.story_agent, story_agent)
    graph_builder.add_edge(agents.connecting_story_agent,agents.story_agent)
    
    return graph_builder

## MAIN AGENT

In [26]:
# PROMPTS

from langchain.prompts import ChatPromptTemplate

multi_system_prompt = """

You do not have any knowledge of Mathematics and stories. You should only use the information provided by the assistants
 Ask for user what things needs to be done.
 
Do not perform activities that are not directly supported by the defined agents.
Always call the checking tool to check if the generated answer is valid.
"""
multi_agent_prompt = ChatPromptTemplate.from_messages(
[(
            "system",
            multi_system_prompt,
        ),
        ("placeholder", "{messages}")])

In [27]:
#CHAINS

def multi_agent_chain(llm):
    return(
        multi_agent_prompt | llm.bind_tools(
            [
                check,
                ToNumericAssistant,
                ToStoryAssistant,
            ],
            parallel_tool_calls = False
    ))

In [28]:
# DEFINING AGENTS

def multi_agent(state: State):
    response = multi_agent_chain(llm).invoke({"messages":state["messages"]})
    return {"messages":[response]}

In [29]:
# ROUTER

def route_agent(state: State):
    route = tools_condition(state)
    if route == END:
        return END
    tool_calls = state["messages"][-1].tool_calls
    if tool_calls:
        tool_name = tool_calls[0]["name"]
        if tool_name == ToNumericAssistant.__name__:
            return agents.connecting_num_agent
        elif tool_name == ToStoryAssistant.__name__:
            return agents.connecting_story_agent
        elif tool_name == "check":
            return agents.checking_tools
            

In [30]:
# GRAPH
graph_builder = StateGraph(State)
graph_builder.add_node(agents.multi_agent, multi_agent)
graph_builder.add_edge(START, agents.multi_agent)
graph_builder.add_conditional_edges(agents.multi_agent, route_agent)
graph_builder.add_node(agents.leave_skill, pop_dialog_state)
graph_builder.add_edge(agents.leave_skill, agents.multi_agent)
graph_builder.add_node(agents.checking_tools, ToolNode(tools = [check]))
graph_builder.add_edge(agents.checking_tools, agents.multi_agent)


graph_builder = numeric_builder(graph_builder,llm)
graph_builder = story_builder(graph_builder,llm)

graph = graph_builder.compile()



In [31]:
# OUTPUT
while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break
    config = { "configurable": { "thread_id": "1"}}
    events = graph.stream({"messages": [("user", user_input)]},config,
    stream_mode="values")
    
    for event in events:
        if "messages" in event:
            event["messages"][-1].pretty_print() 


User:  what is 4!


================================ Human Message =================================

what is 4!
================================== Ai Message ==================================
Tool Calls:
  ToNumericAssistant (call_dWcFAR1JE8FLDcB4AQirB7nM)
 Call ID: call_dWcFAR1JE8FLDcB4AQirB7nM
  Args:
    request: Calculate 4! (4 factorial)
    reason: Factorial calculation is a numeric operation.
================================= Tool Message =================================

The assistant is now the Tool message before going to numeric_agent. Reflect on the above conversation
            between the host assistant and the user.The user's intent is unsatisfied.
            Use the provided tools to assist the user. Remember, you are Tool message before going to numeric_agent,
            If the user is trying to perform operations that are not supported by you
            then call the CompleteOrEscalate. If the user changes their mind or needs
            help for other tasks, call the CompleteOrEs

User:  create a story


================================ Human Message =================================

create a story
================================== Ai Message ==================================
Tool Calls:
  ToStoryAssistant (call_ZdsmAzEdgKzrQ9SxSpDpSZlQ)
 Call ID: call_ZdsmAzEdgKzrQ9SxSpDpSZlQ
  Args:
    request: Please create a story.
    reason: The user requested a story to be created.
================================= Tool Message =================================

The assistant is now the Tool message before going to story_agent. Reflect on the above conversation
            between the host assistant and the user.The user's intent is unsatisfied.
            Use the provided tools to assist the user. Remember, you are Tool message before going to story_agent,
            If the user is trying to perform operations that are not supported by you
            then call the CompleteOrEscalate. If the user changes their mind or needs
            help for other tasks, call the CompleteOrEscalate fun

User:  create a sad ending with 15 lines of story


================================ Human Message =================================

create a sad ending with 15 lines of story
================================== Ai Message ==================================
Tool Calls:
  ToStoryAssistant (call_iJF6Lt3NQUArUUJaAEwmMd1k)
 Call ID: call_iJF6Lt3NQUArUUJaAEwmMd1k
  Args:
    request: create a sad ending with 15 lines of story
    reason: The user requested a story with a sad ending
================================= Tool Message =================================

The assistant is now the Tool message before going to story_agent. Reflect on the above conversation
            between the host assistant and the user.The user's intent is unsatisfied.
            Use the provided tools to assist the user. Remember, you are Tool message before going to story_agent,
            If the user is trying to perform operations that are not supported by you
            then call the CompleteOrEscalate. If the user changes their mind or needs
            he

User:  create a sad ending with 5 lines


================================ Human Message =================================

create a sad ending with 5 lines
================================== Ai Message ==================================
Tool Calls:
  ToStoryAssistant (call_yQZ8ky3ng30R8fe4iBOnSZ0x)
 Call ID: call_yQZ8ky3ng30R8fe4iBOnSZ0x
  Args:
    request: create a sad ending with 5 lines
    reason: The user requested a sad ending for a story.
================================= Tool Message =================================

The assistant is now the Tool message before going to story_agent. Reflect on the above conversation
            between the host assistant and the user.The user's intent is unsatisfied.
            Use the provided tools to assist the user. Remember, you are Tool message before going to story_agent,
            If the user is trying to perform operations that are not supported by you
            then call the CompleteOrEscalate. If the user changes their mind or needs
            help for other tasks, 

User:  q


Goodbye!
